# Deep Residual with Depthwise Separable Convolutions

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
from tqdm import tqdm
import time
import random
random.seed(722)

## Preprocess Data

In [ ]:
def prepare_trainset(trainset):
  output = []
  for img_tensors, label in tqdm(trainset, desc="Processing"):
    data = []
    for z in img_tensors:
      d = []
      for i in range(len(z)):
        if i == 0 and i == len(z)-1:
          nt = z[i]
        elif i+1 < len(z):
          # print(f'hello: {z[i]}')
          nt = torch.stack([z[i-1], z[i], z[i+1]])
          nt = torch.mean(nt, dim=0)
        else:
          #print(z[i])
          nt = z[i]
        d.append(nt)
      d = torch.stack(d)
      data.append(d)
    data = torch.stack(data)
    output.append((data, label))
  return output
# print(data[2][31])

### Data preparation

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# self.transform = transforms.Compose([transforms.Resize(256),
#                                      transforms.CenterCrop(224),
#                                      transforms.ToTensor(),
#                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#                                      ])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainset = prepare_trainset(trainset)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testset = prepare_trainset(testset)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


Processing: 100%|██████████| 50000/50000 [03:35<00:00, 232.28it/s]


Files already downloaded and verified


Processing: 100%|██████████| 10000/10000 [00:43<00:00, 231.15it/s]


In [ ]:
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from torch.nn import init
import torch

__all__ = ['xception']

model_urls = {
    'xception':'http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth'
}


class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
        self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)

    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x


class Block(nn.Module):
    def __init__(self,in_filters,out_filters,reps,strides=1,start_with_relu=True,grow_first=True):
        super(Block, self).__init__()

        if out_filters != in_filters or strides!=1:
            self.skip = nn.Conv2d(in_filters,out_filters,1,stride=strides, bias=False)
            self.skipbn = nn.BatchNorm2d(out_filters)
        else:
            self.skip=None

        self.relu = nn.ReLU(inplace=True)
        rep=[]

        filters=in_filters
        if grow_first:
            rep.append(self.relu)
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))
            filters = out_filters

        for i in range(reps-1):
            rep.append(self.relu)
            rep.append(SeparableConv2d(filters,filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(filters))

        if not grow_first:
            rep.append(self.relu)
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))

        if not start_with_relu:
            rep = rep[1:]
        else:
            rep[0] = nn.ReLU(inplace=False)

        if strides != 1:
            rep.append(nn.MaxPool2d(3,strides,1))
        self.rep = nn.Sequential(*rep)

    def forward(self,inp):
        x = self.rep(inp)

        if self.skip is not None:
            skip = self.skip(inp)
            skip = self.skipbn(skip)
        else:
            skip = inp

        x+=skip
        return x



class Xception(nn.Module):
    """
    Xception optimized for the ImageNet dataset, as specified in
    https://arxiv.org/pdf/1610.02357.pdf
    """
    def __init__(self, num_classes=10):
        """ Constructor
        Args:
            num_classes: number of classes
        """
        super(Xception, self).__init__()


        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(3, 32, 3,2, 0, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(32,64,3,bias=False)
        self.bn2 = nn.BatchNorm2d(64)

        # self.downsample = nn.Conv2d()
        #do relu here

        self.block1=Block(64,128,2,2,start_with_relu=False,grow_first=True)
        self.block2=Block(128,256,2,2,start_with_relu=True,grow_first=True)
        self.block3=Block(256,512,2,2,start_with_relu=True,grow_first=True)
        # block with 3 filter
        self.block4=Block(512,512,3,1,start_with_relu=True,grow_first=True)
        self.block5=Block(512,512,3,1,start_with_relu=True,grow_first=True)
        self.block6=Block(512,512,3,1,start_with_relu=True,grow_first=True)
        self.block7=Block(512,512,3,1,start_with_relu=True,grow_first=True)

        #self.block8=Block(256,256,3,1,start_with_relu=True,grow_first=True)
        #self.block9=Block(256,256,3,1,start_with_relu=True,grow_first=True)
        #self.block10=Block(256,256,3,1,start_with_relu=True,grow_first=True)
        #self.block11=Block(256,256,3,1,start_with_relu=True,grow_first=True)

        # Adding conv layer in mid block
        self.conv_m = nn.Conv2d(512, 512, 1, 1,bias=False)
        self.bn_m = nn.BatchNorm2d(512)
        self.conv_m2 = nn.Conv2d(512,512,1,bias=False)
        # self.bn_m2 = nn.BatchNorm2d(512)

        self.block12=Block(512,768,3,2,start_with_relu=True,grow_first=False)

        self.conv3 = SeparableConv2d(768, 896,3,1,1)
        self.bn3 = nn.BatchNorm2d(896)

        #do relu here
        self.conv4 = SeparableConv2d(896,1024,3,1,1)
        self.bn4 = nn.BatchNorm2d(1024)

        self.fc = nn.Linear(1024, num_classes)



        #------- init weights --------
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        #-----------------------------





    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)

        residual = x
        x = self.block4(x)
        x += residual
        x = self.relu(x)
        residual = x
        x = self.block5(x)
        x += residual
        x = self.relu(x)
        x = self.block6(x)
        residual = x
        x = self.block7(x)
        x += residual
        x = self.relu(x)
        # Start of Residual Block
        residual = x
        #x = self.block8(x)
        #x = self.block9(x)
        #x = self.block10(x)
        #x = self.block11(x)
        x = self.conv_m(x)
        x = self.bn_m(x)
        x += residual
        residual = x
        x = self.relu(x)
        x = self.conv_m2(x)
        x = self.bn_m(x)
        x += residual
        x = self.relu(x)

        x = self.block12(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)

        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def xception(pretrained=False,**kwargs):
    """
    Construct Xception.
    """

    model = Xception(**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['xception']))
    return model

### Train the network


In [ ]:

def train(model, trainloader, epoch, optimizer, criterion, device):
    time_logs = []
    loss_logs = []
    mem_usage = []
    start_time = time.time()

    #for epoch in range(epochs):  # loop over the dataset multiple times
    #time_logs[epoch] = []
    #loss_logs[epoch] = []

    running_loss = 0.0
    time_intv = 1#len(trainloader) / 5
    for i, data in enumerate(tqdm(trainloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        #if (i+1) % time_intv == 0:    # print every 2000 mini-batches
        current_loss = loss.item()
        #tqdm.write(f'\n[{epoch + 1}, {(i + 1):5d}] loss: {current_loss:.3f}')
        loss_logs.append(current_loss)
        time_logs.append(time.time() - start_time)
        mem_usage.append(torch.cuda.memory_allocated())
        #running_loss = 0.0
    tqdm.write(f'\n[{epoch + 1}, {(i + 1):5d}] loss: {running_loss/len(trainloader):.3f}')
    return model, time_logs, loss_logs, mem_usage


### Evaluation

In [ ]:
def evaluate(model, testloader, criterion, device):
    time_logs = 0.0
    loss_logs = 0.0
    start_time = time.time()
    total_loss = 0.0
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss_logs = total_loss / len(testloader)
    time_logs = time.time() - start_time
    return time_logs, loss_logs, 100 * (correct / total)


### Training and Validate the model

In [ ]:
torch.cuda.memory_allocated()

0

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = xception()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-5) #optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f'Num of total params: {sum(p.numel() for p in model.parameters())}')
print(f'Num of trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

train_time_logs = {}
train_loss_logs = {}
train_mem_usage = {}

test_time_logs = {}
test_loss_logs = {}
test_acc_logs = {}
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model, time_logs, loss_logs, mem_usage = train(model, trainloader, epoch, optimizer, criterion, device)
    train_time_logs[epoch] = time_logs
    train_loss_logs[epoch] = loss_logs
    train_mem_usage[epoch] = mem_usage
    time_logs, loss_logs, accuracy = evaluate(model, testloader, criterion, device)
    test_time_logs[epoch] = time_logs
    test_loss_logs[epoch] = loss_logs
    test_acc_logs[epoch] = accuracy
    #print('Epoch {}/{}: Train Loss = {:.4f}, Accuracy = {:.4f}, Dev Loss = {:.4f}, Dev Accuracy = {:.4f}'.format(epoch+1, num_epochs, train_loss, acc, dev_loss, accuracy))
    # test_prediction = evaluate(model, test_dataset, device, epoch, test_data_file)

Num of total params: 7428074
Num of trainable params: 7428074


100%|██████████| 1563/1563 [00:41<00:00, 37.92it/s]



[1,  1563] loss: 1.781


100%|██████████| 1563/1563 [00:41<00:00, 37.77it/s]



[2,  1563] loss: 1.427


...

100%|██████████| 1563/1563 [00:41<00:00, 37.60it/s]



[9,  1563] loss: 0.627


100%|██████████| 1563/1563 [00:43<00:00, 36.25it/s]



[10,  1563] loss: 0.541


100%|██████████| 313/313 [00:03<00:00, 94.23it/s]


In [ ]:
print(test_time_logs)
print(test_loss_logs)
print(test_acc_logs)

{0: 3.3914635181427, 1: 4.0495500564575195, 2: 4.089480400085449, 3: 3.2340753078460693, 4: 4.113281726837158, 5: 3.761772871017456, 6: 3.4084720611572266, 7: 4.0168633460998535, 8: 3.702141761779785, 9: 3.331998825073242}
{0: 1.54989404762134, 1: 1.422701166079829, 2: 1.3639029235885547, 3: 1.3398487775470502, 4: 1.3383486427057285, 5: 1.3533265733490356, 6: 1.3822588998669634, 7: 1.4369173276538667, 8: 1.4790992361668962, 9: 1.5321042989008724}
{0: 44.48, 1: 49.46, 2: 51.459999999999994, 3: 52.75, 4: 53.32, 5: 53.94, 6: 53.59, 7: 53.09, 8: 52.87, 9: 52.800000000000004}


In [ ]:
# print(train_time_logs)
# print(train_loss_logs)
# print('test stats')
# print(test_time_logs)
# print(test_loss_logs)
# print(test_acc_logs)
import json

js_data = {
    'train_time_logs': train_time_logs,
    'train_loss_logs': train_loss_logs,
    'train_mem_usage': train_mem_usage,
    'test_time_logs': test_time_logs,
    'test_loss_logs': test_loss_logs,
    'test_acc_logs': test_acc_logs
}

with open('optimized_stats.processed.json', 'w') as f:
  json.dump(js_data, f, ensure_ascii=False)